In [ ]:
#import the necessary libraries
import glob, pylab, pandas as pd
import pydicom, numpy as np
import cv2
from matplotlib.pyplot import imshow
import matplotlib.pyplot as plt
from sklearn.utils import shuffle
import os
import sys
import scipy.io
import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow
from PIL import Image
import numpy as np
import tensorflow as tf
import imageio
import h5py
import tables
from decimal import Decimal
import time
import functools
from functools import reduce
import cv2
import csv
import imageio
%matplotlib inline

In [ ]:
#this ensures the program can use all the gpu resources it can get
config = tf.ConfigProto()
config.gpu_options.allow_growth=True
sess = tf.Session(config=config)

In [ ]:
batch_size = 1

In [ ]:
# initialize weights
def weight_initializer(weight_input, output_channel_size, filter_size): 
    
    _, rows, columns, input_channel_size = [i.value for i in weight_input.get_shape()]
    
    weight_shape = [filter_size,filter_size,input_channel_size,output_channel_size]

    weight_output = tf.Variable(tf.contrib.layers.xavier_initializer(uniform = False)(weight_shape))
        
    return weight_output

In [ ]:
# convolution block 
def conv2d(block_input, num_filters, filter_size = 1, stride_length = 1): 
    
    init_weights = weight_initializer(block_input, num_filters, filter_size) 
    strides = [1,stride_length,stride_length,1]
    block_output = tf.nn.conv2d(block_input,init_weights,strides,padding='VALID')
    
    return block_output

In [ ]:
def conv_bn_relu(block_input, num_filters, filter_size = 1, stride_length = 1): 
    
    init_weights = weight_initializer(block_input, num_filters, filter_size) 
    strides = [1,stride_length,stride_length,1]
    
    block_output = tf.nn.conv2d(block_input,init_weights,strides,padding='VALID')
    normalized = tf.contrib.layers.batch_norm(block_output, 0.9, epsilon=1e-5, activation_fn = tf.nn.relu)
    
    return normalized

In [ ]:
def conv_block(block_input, num_filters):
    norm_1 = tf.contrib.layers.batch_norm(block_input, 0.9, epsilon=1e-5, activation_fn = tf.nn.relu)
    conv_1 = conv2d(norm_1, int(num_filters/2), 1, 1)
    norm_2 = tf.contrib.layers.batch_norm(conv_1, 0.9, epsilon=1e-5, activation_fn = tf.nn.relu)
    pad = tf.pad(norm_2, np.array([[0,0],[1,1],[1,1],[0,0]]))
    conv_2 = conv2d(pad, int(num_filters/2), 3, 1)
    norm_3 = tf.contrib.layers.batch_norm(conv_2, 0.9, epsilon=1e-5, activation_fn = tf.nn.relu)
    conv_3 = conv2d(norm_3, int(num_filters), 1, 1)
    
    return conv_3

In [ ]:
def skip_layer(block_input, num_filters):
    
    if (block_input.get_shape()[3] == num_filters):
        return block_input
    else:
        conv = conv2d(block_input, num_filters,1,1)
        return conv

In [ ]:
def residual(block_input, num_filters):
    conv = conv_block(block_input, num_filters)
    skip = skip_layer(block_input, num_filters)
    
    return(tf.add_n([conv,skip]))

In [ ]:
def hourglass_unit(input_data, reduction_factor, num_filters):
    up_1 = residual(input_data, num_filters)
    low = tf.contrib.layers.max_pool2d(input_data, [2,2],[2,2], 'VALID')
    low_1 = residual(low, num_filters)
    
    if reduction_factor > 0:
        low_2 = hourglass_unit(low_1, reduction_factor - 1, num_filters)
    else:
        low_2 = residual(low_1, num_filters)
    
    low_3 = residual(low_2, num_filters)
    up_sample = tf.image.resize_nearest_neighbor(low_3, tf.shape(low_3)[1:3]*2)
    return tf.add_n([up_1, up_sample])

In [ ]:
def hourglass_model(input_data, num_blocks, num_filters, reduction_factor, train_model):
    pad_1 = tf.pad(input_data, np.array([[0,0],[2,2],[2,2],[0,0]]))
    conv_1 = conv2d(pad_1, 64,6,2)
    res_1 = residual(conv_1, 128)
    pool_1 = tf.contrib.layers.max_pool2d(res_1, [2,2], [2,2], padding= 'VALID')
    res_2 = residual(pool_1, 128)
    res_3 = residual(res_2, num_filters)
    
    x1 = [None] * num_blocks
    x2 = [None] * num_blocks
    x3 = [None] * num_blocks
    x4 = [None] * num_blocks
    x5 = [None] * num_blocks
    x6 = [None] * num_blocks
    sum_all = [None] * num_blocks
    
    x1[0] = hourglass_unit(res_3, reduction_factor, num_filters)
    x2[0] = conv_bn_relu(x1[0], num_filters)
    x3[0] = conv2d(x2[0], num_filters, 1, 1)
    x4[0] = tf.layers.dropout(x3[0], rate = 0.1, training = train_model)
    x5[0] = conv2d(x2[0], num_filters, 1, 1)
    x6[0] = conv2d(x5[0], num_filters, 1, 1)
    sum_all[0] = tf.add_n([x4[0], x6[0], res_3])
    
    for i in range(1, num_blocks - 1):
        x1[i] = hourglass_unit(sum_all[i-1], reduction_factor, num_filters)
        x2[i] = conv_bn_relu(x1[i], num_filters)
        x3[i] = conv2d(x2[i], num_filters, 1, 1)
        x4[i] = tf.layers.dropout(x3[i], rate = 0.1, training = train_model)
        x5[i] = conv2d(x2[i], num_filters, 1, 1)
        x6[i] = conv2d(x5[i], num_filters, 1, 1)
        sum_all[i] = tf.add_n([x4[i], x6[i], sum_all[i-1]])
    
    x1[num_blocks - 1] = hourglass_unit(sum_all[num_blocks - 2], reduction_factor, num_filters)
    x2[num_blocks - 1] = conv_bn_relu(x1[num_blocks - 1], num_filters)
    x4[num_blocks - 1] = tf.layers.dropout(x2[num_blocks - 1], rate = 0.1, training = train_model)
    x5[num_blocks - 1] = conv2d(x4[num_blocks - 1], 3, 1, 1)
    final_output = tf.image.resize_nearest_neighbor(x5[num_blocks - 1], tf.shape(x5[num_blocks - 1])[1:3]*2)
    return final_output

In [ ]:
sess = tf.InteractiveSession()

MODEL LOADING BLOCK -> The code block below is where the model weights are loaded. Select the model you want to load. The other model must be loaded after the intermediate csv file is generated.

In [ ]:
lung_x_input = tf.placeholder(tf.float32,shape=(batch_size,256,256,3),name='pose_img_ip')
###################################################################################################################

lung_x_input = lung_x_input/255.0

hg_output = hourglass_model(lung_x_input, 4, 256, 3, False) # true while training, false during inference
###################################################################################################################

sess.run(tf.global_variables_initializer())

###################################################################################################################

restore_model = True

#restore model weights. 
if(restore_model):
    saver =  tf.train.Saver()  
    saver.restore(sess,'../models/lungs_3_5') #change the model name here


INTERMEDIATE CSV BLOCK - After the model weights are loaded from the block above, you can use the code block below to generate the intermediate csv file. You can change the csv file name and the pixel threshold values below. 

for model lungs_4_2, use threshold value of 50 and csv file name of submission_28.csv
for model lungs_3_5, use threshold value of 60 and csv file name of submission_50.csv

In [ ]:
#this is for generating the csv file

df_test = pd.read_csv('../data/stage_1_sample_submission.csv')

with open('../output/submission_28.csv', 'w', newline='') as csvfile: #CHANGE THE CSV FILE NAME HERE
    csvwriter = csv.writer(csvfile, delimiter=',', quoting=csv.QUOTE_NONE)

    for i in range(df_test.shape[0]): #df_test.shape[0]
        patientId = df_test['patientId'][i]
        dcm_file = '../data/stage_1_test_images/%s.dcm' % patientId
        dcm_data = pydicom.read_file(dcm_file)
        im = dcm_data.pixel_array
        im = np.stack([im] * 3, axis=2)
        original_img = im
        im = cv2.resize(im, (256,256), interpolation = cv2.INTER_AREA)
        temp_content = im.reshape(1,256,256,3)

        temp_outputs = sess.run([hg_output],feed_dict={lung_x_input:temp_content})
        temp_outputs = np.asarray(temp_outputs)

        gen_img = np.clip(temp_outputs[0,0,:,:,:], 0, 255).astype('uint8')
        
        gray = cv2.cvtColor(gen_img, cv2.COLOR_RGB2GRAY)
        
        #submission_28.csv THRESHOLD VALUE 50 area val 256. score - 0.134. model used - lungs_4_2
        #submission_50.csv THRESHOLD VALUE 60 area 256 score - 0.126 model used - lungs_3_5
        
        thresh_val = np.array([50]) #YOU CAN CHANGE THE PIXEL THRESHOLD VALUE HERE (CHANGE 50 TO 60 OR VICE VERSA).
        selected_thresh_val = 0
        val_is_good = False
        
        for j in range(thresh_val.shape[0]):
            gray_2 = cv2.threshold(gray, thresh_val[j], 255, cv2.THRESH_BINARY)[1]
            selected_thresh_val = thresh_val[j]
            output = cv2.connectedComponentsWithStats(gray_2, 8, cv2.CV_32S)
            num_labels = output[0]
            stats = output[2]
                
        if num_labels > 1:
            box_text = ""
            for m in range(stats.shape[0]):
                if m > 0 and stats[m,4] > 256:              
                    box_text = box_text + "1.0 " + str(stats[m,0]*8) + " " + str(stats[m,1]*8) + " " + str(stats[m,2]*8) + " " + str(stats[m,3]*8) + " "

            #print(box_text)
            csvwriter.writerow([patientId, box_text])
        else:
            csvwriter.writerow([patientId, ""])
        
        
        

FINAL CSV BLOCK -> To generate the final csv file for submission, I combine the two intermediate csv files generated from the two models. Generate the two intermediate files above by loading the two models separately. Then run the code below.

In [ ]:
#we then simply combine the csv output of the models below to generate the final result.
df_left = pd.read_csv('../output/submission_50.csv')
df_right = pd.read_csv('../output/submission_28.csv')

with open('../output/submission_combo_1.csv', 'w', newline='') as csvfile:
    csvwriter = csv.writer(csvfile, delimiter=',', quoting=csv.QUOTE_NONE)

    for i in range(df_left.shape[0]):
        patientId = df_left['patientId'][i]
        data_left = df_left['PredictionString'][i]
        data_right = df_right['PredictionString'][i]
                
        #if data on left is more than right copy the whole data over
        if not pd.isnull(data_right) and not pd.isnull(data_left):
            len_right = len(data_right.split(" "))
            len_left = len(data_left.split(" "))
            
            if len_left > len_right:
                data_right = data_left
            
        #lets try - if data on left and no data on right, copy over
        if pd.isnull(data_right) and not pd.isnull(data_left):
            data_right = data_left   
            
        if pd.isnull(data_right):
            data_right = ""
            
        csvwriter.writerow([patientId, data_right])